In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#define the function for which the integral will be performed, through trapzoid, simpson's, and romberg methods
def func(x):
    a = -2
    b = 10
    return (np.exp(a*x))*(np.cos(b*x)) #function = e^(-2x)*cos(10x)
    
#define integral function, to compare to methods
def func_integral(x):
    a = -2
    b = 10
    c = 5
    d = 52
    return(((np.exp(a*x))*((c*np.sin(b*x))-np.cos(b*x)))/d)

In [ ]:
#def core of trapezoid method
def trapezoid_core(f, x, h):
    return 0.5*h*(f(x+h) + f(x)) #area of the trapezoid interval ("core")

#def wrapper function to perform trapezoid method
def trapezoid_method(f, a, b):
    #f = function to integrate 
    # a = lower limit of integral
    # b = upper limit
    # N = number of intervals/steps

    #def x values for performing integral
    x = np.linspace(a, b) #x values in each interval for all intervals
    h = x[1] - x[0] #upper - lower limit = width between the 2 (x) points
    
    #def value of the integral (initially)
    Fint = 0
    
    #perform integral using trapezoid method
    for i in range(0, len(x)-1, 1): # left to just short of right side for every interval, in steps of 1
        Fint += trapezoid_core(f, x[i], h) #go on to next interval
        
    #number of intervals needed to get to answer
    print("number of intervals:", len(x))
    
    #return answer of integral, adding all together
    return Fint 
    

In [ ]:
#define the core of the simpson's method
def simpsons_core(f, x, h):
    return h* (f(x) + 4*f(x+h) + f(x+2*h))/3 #adding right, middle, and left of the interval, in a weighted sum

#define the wrapper function to perform the method over all intervals
def simpsons_method(f, a, b):
    x = np.linspace(a, b)
    h = x[1] - x[0]

    Fint = 0.0 #same as before
    
    for i in range(0, len(x)-2, 2):
        Fint += simpsons_core(f, x[i], h)
        
    #apply simpsons rule over the last interval, if N is even
    
    if((len(x)%2)==0):
        Fint +=simpsons_core(f, x[-2], 0.5*h)
        
    #print number of intervals to get to answer
    print("number of intervals:", len(x))
    
    return Fint

In [ ]:
#def Romberg integration --> calls trapezoid method many times, in successively refined grids
def romberg_core(f, a, b, i):
    h = b-a #difference b-a (upper - lower limit)
    
    #incremement between new func evals
    dh = h/2.**i #delta h  = h/2^i
    
    #cofactor
    K = h/2.**(i+1)
    
    #function evals
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
    
    #return answer
    return K*M

#define wrapper function - which uses tolerance to perform romberg integration w/ core grids 
#integration iterates with refined grids until it reaches tolerance
def romberg_integration(f, a, b, tol):
    #def iterator
    i = 0
    
    #def max number of iterations
    imax = 1000
    
    #def error estimate, set to a large value -error should be bigger than absolute value tolerance
    delta = 100*np.fabs(tol)
    
    #set array of integral answers
    I = np.zeros(imax, dtype=float)
    
    #get zeroth romberg integration
    I[0] = 0.5*(b-a)*(f(a)+f(b))

    #iterate by 1
    i +=1
    
    #while loop
    while (delta>tol):
        #find this romberg integration:
        I[i] = 0.5*I[i-1] + romberg_core(f, a, b, i)
        
        #compute new fractional error estimate
        delta = np.fabs((I[i] - I[i-1])/I[i])
        
        print(i, I[i], I[i-1], delta)
        
        if(delta>tol):
            #iterate
            i +=1
            
            #if we've reached max iterations
            if (i>imax):
                print("Max iterations reached")
                raise StopIteration('stopping iterations after', i)
      

    #print number of iterations to get to answer
    print("number of iterations:", i)
    #outside while loop, return answer
    return I[i]


In [ ]:
#check the integrals
Answer = func_integral(np.pi) - func_integral(0)
print("true answer:", Answer)

print("trapezoid method")
print("answer:",trapezoid_method(func, 0, np.pi))

print("simpson's method")
print("answer:",simpsons_method(func, 0, np.pi))

print("romberg integration")
tolerance = 1.0e-4
RI = romberg_integration(func, 0, np.pi, tolerance)
print("answer:",RI, "/fractional error:",(RI-Answer)/Answer, "/tolerance:",tolerance)